# **INTRODUCTIONS**

---

This flow is added because of a depencency to ECDC_VARIANTS



1. **[Tables](#tables)**
1. **[Datatino configurations](#DATATINO-CONFIGURATIONS)**


# **DEPENDENCIES**
---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb"
    ]
}
```

# **TABLES**
---

### STAGING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

-- 1) CREATE TABLE(S)..... 
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_VACCINATION_SERIES_MASTER_DATA]'))
BEGIN
  CREATE TABLE [VWSSTAGE].[RIVM_VACCINATION_SERIES_MASTER_DATA] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [VERSION] VARCHAR (100) NULL,
    [SERIES_START_DATE] VARCHAR (100) NULL,
    [SERIES_END_DATE] VARCHAR(100) NULL,
    [LABEL_NL] VARCHAR (100) NULL,
    [LABEL_EN] VARCHAR (100) NULL,
    [SORT_ORDER] VARCHAR (100) NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT (GETDATE()) NULL
  );
END

GO

### STATICS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordination for more information.

-- 1) CREATE TABLE(S).....

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA]'))
BEGIN
  CREATE TABLE [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA] (
    [ID] BIGINT PRIMARY KEY IDENTITY(1,1),
    [VERSION] INT NULL,
    [SERIES_START_DATE]  DATETIME NULL,
    [SERIES_END_DATE]  DATETIME NULL,
    [LABEL_NL] VARCHAR (100) NULL,
    [LABEL_EN] VARCHAR (100) NULL,
    [SORT_ORDER] INT NULL,
    [DATE_LAST_INSERTED] DATETIME DEFAULT (GETDATE()) NULL
  );
END

GO


# **STORED PROCEDURES**
---

### STAGE &rarr; STATIC MAPPING

In [ ]:
-- 1) CREATE STORED PROCEDURE(S).....
CREATE OR ALTER PROCEDURE [DBO].[SP_RIVM_VACCINATION_SERIES_MASTER_DATA_STAGING_TO_STATIC]
AS
BEGIN
        INSERT INTO [VWSSTATIC].[RIVM_VACCINATION_SERIES_MASTER_DATA] (
            [VERSION],
            [SERIES_START_DATE],
            [SERIES_END_DATE],
            [LABEL_NL],
            [LABEL_EN],
            [SORT_ORDER]
        )
        SELECT
            CAST([VERSION] as INT),
            CAST([SERIES_START_DATE] as DATETIME),
            CAST([SERIES_END_DATE] as DATETIME),
            [LABEL_NL],
            [LABEL_EN],
            CAST([SORT_ORDER] as INT)
        FROM [VWSSTAGE].[RIVM_VACCINATION_SERIES_MASTER_DATA]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_VACCINATION_SERIES_MASTER_DATA])
END;
GO

# **DATATINO CONFIGURATIONS**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'RIVM_VACCINATION_SERIES_MASTER_DATA',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;


    -- 1.1) SET ENVIRONMENTAL VARIABLES.....
    SET @is_active = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 1
        WHEN 'acceptance' THEN 1
        ELSE 1
    END;

    SELECT TOP(1)
        @workflow_id = workflows.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
    WHERE v_workflows.[NAME] = @workflow_name;

    SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
        @id = @workflow_id, 
        @workflow_name = @workflow_name,
        @description = @workflow_description,
        @schedule = '0 12 * * FRI', -- AT 14:00 PM (DUTCH SUMMER TIME), ONLY ON FRIDAYS  
        @active = @is_active;

    -- 2) UPSERT SOURCE(S).....
    DECLARE @source NVARCHAR(256),
            @source_id INT,
            @source_name NVARCHAR(256),
            @source_description NVARCHAR(256),
            @target_name VARCHAR(256),
            @location_type VARCHAR(50),
            @security_profile VARCHAR(50);

    -- 2.1) SET ENVIRONMENTAL VARIABLES.....
    SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/STATIC/RIVM_VACCINATION_SERIES_MASTER_DATA.csv'
    WHEN 'acceptance' THEN 'datafiles/STATIC/RIVM_VACCINATION_SERIES_MASTER_DATA.csv'
        ELSE 'datafiles/STATIC/RIVM_VACCINATION_SERIES_MASTER_DATA.csv'
    END;

    SET @location_type = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'AzureBlob'
        WHEN 'acceptance' THEN 'AzureBlob'
        ELSE 'AzureBlob'
    END;

    SET @security_profile = CASE LOWER('#{ Environment }#')
        WHEN 'production' THEN 'N/A'
        WHEN 'acceptance' THEN 'N/A'
        ELSE 'N/A'
    END;

    -- 2.2) UPSERT TABLE SOURCE(S): STAGING......
    SET @source_name = CONCAT('SOURCE_', @workflow_name);
    SET @source_description = CONCAT('LOAD: ', @workflow_name);
    SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

    SELECT TOP(1)
        @source_id = [ID]
    FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
    WHERE [NAME] = @source_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
        @id = @source_id,
        @source_name = @source_name,
        @description = @source_description,
        @source = @source,
        @source_columns = 'Version|Series_start_date|Series_end_date|Label_NL|Label_EN|Sort_order',
        @target_columns = 'Version|Series_start_date|Series_end_date|Label_NL|Label_EN|Sort_order',
        @target_name = @target_name,
        @source_type = 'CsvFile',
        @location_type = @location_type,
        @delimiter_type = 'SemiColon',
        @security_profile= @security_profile;

    -- 2) UPSERT PROCESS(ES).....
    DECLARE @process_id INT,
            @process_name NVARCHAR(256),
            @process_description NVARCHAR(256),
            @process_source_name VARCHAR(256);

    -- 2.1) UPSERT TABLE PROCESS(ES): STAGING......
    SET @process_name = CONCAT('PROCESS_', @workflow_name);
    SET @process_description = CONCAT('LOAD: ', @workflow_name);
    SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

    SELECT TOP(1)
        @process_id = processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] = @process_name 
        AND v_workflows.[NAME] = @workflow_name;

    EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
        @id = @process_id,
        @process_name = @process_name,
        @description = @process_description,
        @source_name = @process_source_name,
        @schedule = '* * * * *',
        @workflow_name = @workflow_name,
        @active = 1;

### UPSERT STORED PROCEDURE STAGING TO STATIC

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- 1 UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @workflow_name NVARCHAR(50) = 'RIVM_VACCINATION_SERIES_MASTER_DATA';
        

SET @source = 'DBO.SP_RIVM_VACCINATION_SERIES_MASTER_DATA_STAGING_TO_STATIC';
SET @source_name = 'SOURCE_Staging_Masterdata_RIVM_VACCINATION_SERIES';
SET @source_description = 'Move masterdata RIVM_VACCINATION_SERIES data from stage table to static table';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= 'N/A';

-- 2) UPSERT STORED PROCEDURE PROCESS(S): STAGING TO STATIC......
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);


SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGING_TO_STATIC');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGING TO STATIC');
SET @process_source_name =  @source_name --CONCAT('SOURCE_SP_', @workflow_name, '_STAGING_TO_STATIC');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES)....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGING_TO_STATIC');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGING_TO_STATIC');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;